# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPY/weather_data.csv")
cities_df

,Unnamed: 0,City,Latitude,Longitude,Humidity,Max Temp,Cloudiness,Wind Speed,Country,Date
0,0,hobart,-42.8794,147.3294,35,71.60,75,9.22,AU,1619840524
1,1,fortuna,40.5982,-124.1573,100,53.01,1,3.38,US,1619840507
2,2,upernavik,72.7868,-56.1549,76,23.07,100,4.32,GL,1619840525
3,3,faanui,-16.4833,-151.7500,69,81.45,91,9.33,PF,1619840362
4,4,seabra,-12.4186,-41.7703,100,68.00,90,4.61,BR,1619840525
...,...,...,...,...,...,...,...,...,...,...
497,497,valdivia,-39.8142,-73.2459,100,55.40,90,6.91,CL,1619840671
498,498,pangnirtung,66.1451,-65.7125,75,29.05,31,7.90,CA,1619840672
499,499,kyzyl-suu,42.3414,78.0056,83,50.72,100,3.31,KG,1619840672
500,500,saldanha,-33.0117,17.9442,81,48.20,0,2.30,ZA,1619840672


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
## Use the Lat and Lng as locations
locations = cities_df[["Latitude", "Longitude"]]

In [5]:
## Use Humidity as the weight
humidity = cities_df["Humidity"].astype(float)

In [6]:
## Plot and add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
## Narrow down the cities to fit weather conditions from ReadMe
narrowed_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80), :]
narrowed_df = narrowed_df.dropna(how= "any")
del narrowed_df["Unnamed: 0"]
narrowed_df

,City,Latitude,Longitude,Humidity,Max Temp,Cloudiness,Wind Speed,Country,Date
22,bin qirdan,35.0363,108.0798,24,79.72,0,4.27,CN,1619840531
34,namibe,-15.1961,12.1522,70,75.13,0,5.50,AO,1619840535
62,huainan,32.6264,116.9969,28,74.70,0,0.85,CN,1619840543
99,saint-leu,-21.1500,55.2833,73,75.20,0,2.10,RE,1619840553
167,sabha,27.0377,14.4283,8,77.07,0,8.93,LY,1619840491
187,luxor,25.6989,32.6421,14,78.80,0,2.30,EG,1619840577
215,saint-pierre,-21.3393,55.4781,73,75.20,0,9.53,RE,1619840587
218,sakakah,29.9697,40.2064,17,75.20,0,9.22,SA,1619840588
308,saint-louis,16.3333,-15.0000,64,73.40,0,6.91,SN,1619840615
323,atar,20.5169,-13.0499,21,77.88,0,6.80,MR,1619840619


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
## Store into variable named hotel_df.
hotel_df = narrowed_df.loc[:,["City", "Country", "Latitude", "Longitude"]]

In [9]:
## Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
22,bin qirdan,CN,35.0363,108.0798,
34,namibe,AO,-15.1961,12.1522,
62,huainan,CN,32.6264,116.9969,
99,saint-leu,RE,-21.1500,55.2833,
167,sabha,LY,27.0377,14.4283,
187,luxor,EG,25.6989,32.6421,
215,saint-pierre,RE,-21.3393,55.4781,
218,sakakah,SA,29.9697,40.2064,
308,saint-louis,SN,16.3333,-15.0000,
323,atar,MR,20.5169,-13.0499,


In [10]:
## Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel", 
          "keyword" : "hotel", 
          "radius" : 5000, 
          "key" : g_key}

In [13]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    ## Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("Could not find result")
        print("-----------------------")
        

Retrieving Results for Index 22: bin qirdan.
Closest hotel in bin qirdan is Binzhou International Garden Hotel （Northwest Gate）.
Retrieving Results for Index 34: namibe.
Closest hotel in namibe is Hotel Chik Chik Namibe.
Retrieving Results for Index 62: huainan.
Closest hotel in huainan is GreenTree Eastern Anhui Huainan Guangchang Road Hotel.
Retrieving Results for Index 99: saint-leu.
Closest hotel in saint-leu is Iloha Seaview Hotel.
Retrieving Results for Index 167: sabha.
Closest hotel in sabha is فندق المواجيد.
Retrieving Results for Index 187: luxor.
Closest hotel in luxor is Sofitel Winter Palace Luxor.
Retrieving Results for Index 215: saint-pierre.
Closest hotel in saint-pierre is Villa Delisle, Hotel & Spa.
Retrieving Results for Index 218: sakakah.
Closest hotel in sakakah is Raoum Inn Hotel.
Retrieving Results for Index 308: saint-louis.
Could not find result
-----------------------
Retrieving Results for Index 323: atar.
Closest hotel in atar is Bab Sahara.
Retrieving Res

In [17]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
22,bin qirdan,CN,35.0363,108.0798,Binzhou International Garden Hotel （Northwest ...
34,namibe,AO,-15.1961,12.1522,Hotel Chik Chik Namibe
62,huainan,CN,32.6264,116.9969,GreenTree Eastern Anhui Huainan Guangchang Roa...
99,saint-leu,RE,-21.1500,55.2833,Iloha Seaview Hotel
167,sabha,LY,27.0377,14.4283,فندق المواجيد
187,luxor,EG,25.6989,32.6421,Sofitel Winter Palace Luxor
215,saint-pierre,RE,-21.3393,55.4781,"Villa Delisle, Hotel & Spa"
218,sakakah,SA,29.9697,40.2064,Raoum Inn Hotel
308,saint-louis,SN,16.3333,-15.0000,
323,atar,MR,20.5169,-13.0499,Bab Sahara


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))